In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import re
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import decomposition
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from math import log

from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving clean.csv to clean.csv
User uploaded file "clean.csv" with length 109323139 bytes


In [0]:
df = pd.read_csv('Task_1/train/train.csv')
N = len(df)
print(N)
df.head()


stop_words = ["а","е","и","ж","м","о","на","не","ни","об","но","он","мне","мои","мож","она","они","оно","мной","много","многочисленное","многочисленная","многочисленные","многочисленный","мною","мой","мог","могут","можно","может","можхо","мор","моя","моё","мочь","над","нее","оба","нам","нем","нами","ними","мимо","немного","одной","одного","менее","однажды","однако","меня","нему","меньше","ней","наверху","него","ниже","мало","надо","один","одиннадцать","одиннадцатый","назад","наиболее","недавно","миллионов","недалеко","между","низко","меля","нельзя","нибудь","непрерывно","наконец","никогда","никуда","нас","наш","нет","нею","неё","них","мира","наша","наше","наши","ничего","начала","нередко","несколько","обычно","опять","около","мы","ну","нх","от","отовсюду","особенно","нужно","очень","отсюда","в","во","вон","вниз","внизу","вокруг","вот","восемнадцать","восемнадцатый","восемь","восьмой","вверх","вам","вами","важное","важная","важные","важный","вдали","везде","ведь","вас","ваш","ваша","ваше","ваши","впрочем","весь","вдруг","вы","все","второй","всем","всеми","времени","время","всему","всего","всегда","всех","всею","всю","вся","всё","всюду","г","год","говорил","говорит","года","году","где","да","ее","за","из","ли","же","им","до","по","ими","под","иногда","довольно","именно","долго","позже","более","должно","пожалуйста","значит","иметь","больше","пока","ему","имя","пор","пора","потом","потому","после","почему","почти","посреди","ей","два","две","двенадцать","двенадцатый","двадцать","двадцатый","двух","его","дел","или","без","день","занят","занята","занято","заняты","действительно","давно","девятнадцать","девятнадцатый","девять","девятый","даже","алло","жизнь","далеко","близко","здесь","дальше","для","лет","зато","даром","первый","перед","затем","зачем","лишь","десять","десятый","ею","её","их","бы","еще","при","был","про","процентов","против","просто","бывает","бывь","если","люди","была","были","было","будем","будет","будете","будешь","прекрасно","буду","будь","будто","будут","ещё","пятнадцать","пятнадцатый","друго","другое","другой","другие","другая","других","есть","пять","быть","лучше","пятый","к","ком","конечно","кому","кого","когда","которой","которого","которая","которые","который","которых","кем","каждое","каждая","каждые","каждый","кажется","как","какой","какая","кто","кроме","куда","кругом","с","т","у","я","та","те","уж","со","то","том","снова","тому","совсем","того","тогда","тоже","собой","тобой","собою","тобою","сначала","только","уметь","тот","тою","хорошо","хотеть","хочешь","хоть","хотя","свое","свои","твой","своей","своего","своих","свою","твоя","твоё","раз","уже","сам","там","тем","чем","сама","сами","теми","само","рано","самом","самому","самой","самого","семнадцать","семнадцатый","самим","самими","самих","саму","семь","чему","раньше","сейчас","чего","сегодня","себе","тебе","сеаой","человек","разве","теперь","себя","тебя","седьмой","спасибо","слишком","так","такое","такой","такие","также","такая","сих","тех","чаще","четвертый","через","часто","шестой","шестнадцать","шестнадцатый","шесть","четыре","четырнадцать","четырнадцатый","сколько","сказал","сказала","сказать","ту","ты","три","эта","эти","что","это","чтоб","этом","этому","этой","этого","чтобы","этот","стал","туда","этим","этими","рядом","тринадцать","тринадцатый","этих","третий","тут","эту","суть","чуть","тысяч"]

def clean(string):
    string = string.lower()
    letters = 'ёйцукенгшщзхъфывапролджэячсмитьбю qwertyuiopasdfghjklzxcvbnm'
    ans = ''
    string = ''.join(c for c in string if c in letters)
    spliced = string.split(' ')
    string = ''.join((morph.parse(word)[0]).normal_form+' ' for word in spliced if len(word) > 1 and not word in stop_words)
    return string[:-1]

for i in range(N):
    df.iloc[i]['text'] = clean(df.iloc[i]['text'])
    if i%10 == 0:
        print(i)

df.sample(frac = 1, random_state = 42).head()
df.to_csv('clean.csv', sep=';', encoding='utf-8')

In [4]:
clean =  pd.read_csv("clean.csv", sep  = ";")
clean.head()

,Unnamed: 0,author,text
0,0,__label__34,страница бумажный версия номер игорь павло авт...
1,1,__label__50,мелочь проплатить культура конец минувший росс...
2,2,__label__6,юношеский низкий высокий закончить спорый пред...
3,3,__label__12,свет упасть тишина родиться жить москва окончи...
4,4,__label__58,полемика черносотенство прошлое перспектива за...


In [0]:
y = [int(string[9:]) for string in clean.author.values] # remove "__label__" from string and convert to int

xtrain, xvalid, ytrain, yvalid = train_test_split(clean.text.values, y, 
                                                  stratify=clean["author"], 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [0]:
tfv = TfidfVectorizer(min_df=3, analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1) 

tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain)
valild1_tfv = tfv.transform(xvalid)

In [0]:
regr = LogisticRegression(C=1000, penalty = 'l2' )
regr.fit(xtrain_tfv, ytrain)
predictions = regr.predict(valild1_tfv)
accuracy_score(predictions, yvalid)


/Users/Admin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Admin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.9283667621776505

In [0]:
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [0]:
token = text.Tokenizer(num_words=None)
#max_len = max([len(s.split()) for s in clean['text']])
max_len = 1000
word_index = token.word_index

vocab_size = len(word_index) + 1

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)




In [24]:
EMBEDDING_DIM = 100

print('Build model...')

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length = max_len))
model.add(GRU(units = 32, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(60))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1000, 100)         100       
_________________________________________________________________
gru_3 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_3 (Dense)              (None, 60)                1980      
_________________________________________________________________
activation_3 (Activation)    (None, 60)                0         
Total params: 14,848
Trainable params: 14,848
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 3136 samples, validate on 349 samples
Epoch 1/100


InvalidArgumentError: ignored

[53,
 6,
 15,
 9,
 11,
 29,
 32,
 48,
 51,
 58,
 12,
 10,
 33,
 28,
 42,
 16,
 25,
 25,
 41,
 16,
 49,
 20,
 45,
 12,
 15,
 35,
 37,
 13,
 7,
 0,
 41,
 25,
 1,
 36,
 14,
 52,
 6,
 3,
 30,
 31,
 28,
 44,
 40,
 14,
 5,
 46,
 47,
 54,
 21,
 22,
 31,
 3,
 57,
 41,
 29,
 56,
 20,
 35,
 9,
 7,
 27,
 24,
 48,
 51,
 28,
 35,
 24,
 0,
 31,
 8,
 24,
 9,
 14,
 49,
 59,
 16,
 34,
 16,
 51,
 59,
 31,
 13,
 7,
 29,
 0,
 16,
 52,
 42,
 12,
 37,
 35,
 0,
 12,
 45,
 12,
 31,
 14,
 57,
 31,
 47,
 15,
 17,
 43,
 33,
 0,
 2,
 47,
 47,
 28,
 57,
 59,
 42,
 52,
 42,
 33,
 4,
 5,
 40,
 14,
 8,
 52,
 57,
 47,
 31,
 14,
 48,
 5,
 33,
 10,
 2,
 36,
 30,
 39,
 41,
 31,
 30,
 47,
 54,
 6,
 20,
 32,
 50,
 24,
 31,
 42,
 40,
 1,
 9,
 40,
 14,
 31,
 24,
 34,
 4,
 39,
 26,
 59,
 38,
 11,
 49,
 14,
 41,
 59,
 49,
 0,
 7,
 54,
 5,
 47,
 14,
 18,
 12,
 17,
 32,
 27,
 40,
 42,
 29,
 8,
 9,
 25,
 9,
 48,
 9,
 44,
 33,
 25,
 52,
 57,
 36,
 40,
 19,
 12,
 41,
 30,
 33,
 30,
 21,
 18,
 1,
 8,
 52,
 10,
 40,
 45,
 41,
 36,
 2